In [1]:
from paddleocr import PaddleOCR,draw_ocr

In [2]:
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory
img_path = '../../EAST/test-images/ctu-hkd.png'

[2024/08/05 14:30:14] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/r079a/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/r079a/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_lengt

In [3]:
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)

[2024/08/05 14:30:17] ppocr DEBUG: dt_boxes num : 4, elapsed : 0.5971972942352295
[2024/08/05 14:30:17] ppocr DEBUG: cls num  : 4, elapsed : 0.02868199348449707
[2024/08/05 14:30:17] ppocr DEBUG: rec_res num  : 4, elapsed : 0.04643988609313965
[[[9.0, 804.0], [75.0, 804.0], [75.0, 840.0], [9.0, 840.0]], ('CTU', 0.9941731095314026)]
[[[27.0, 841.0], [80.0, 841.0], [80.0, 892.0], [27.0, 892.0]], ('R', 0.9883725047111511)]
[[[24.0, 880.0], [89.0, 880.0], [89.0, 916.0], [24.0, 916.0]], ('HKD', 0.996049702167511)]
[[[10.0, 928.0], [110.0, 928.0], [110.0, 961.0], [10.0, 961.0]], ('20MIN', 0.9780788421630859)]
